### Search Engine With Tools And Agents

In [ ]:
'''An API Wrapper is a function that helps you easily use an external API inside LangChain. Instead of calling the API manually every time, 
you create a wrapper (a reusable function or class) that automatically formats requests and processes responses. 
This is useful when integrating external services like:
✅ Weather API (for live weather updates)
✅ Stock Market API (for stock prices)'''

In [1]:
## Tools used here: Arxiv (research paper searching) and Wikipedia
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
## Used the inbuilt tool of wikipedia
# https://python.langchain.com/v0.1/docs/modules/tools/
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Lenovo\\Documents\\Data Science1\\Gen AI\\AgenticAI\\3-Chatbots\\myenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

In [3]:
# https://python.langchain.com/docs/integrations/tools/arxiv/
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [4]:
tools=[wiki,arxiv]

In [5]:
# Integrate tools with RAG
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
embeddings=(
    OllamaEmbeddings(model="llama3.2:1b")  ##by default it ues llama2
)

In [9]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020FD903B990>, search_kwargs={})

In [10]:
# Convert retriever into tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [11]:
tools=[wiki,arxiv,retriever_tool]

In [12]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Lenovo\\Documents\\Data Science1\\Gen AI\\AgenticAI\\3-Chatbots\\myenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000020FDAE17240>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstore

In [13]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
# import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
# openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [ ]:
## Prompt Template
# The LangChain Prompt Hub is a centralized repository where users can create, share, and reuse high-quality prompts for different LLM 
# applications. It helps standardize and optimize prompts for various tasks like Q&A, summarization, translation, code generation, and more.
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\Lenovo\Documents\Data Science1\Gen AI\AgenticAI\3-Chatbots\myenv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
'''
The OpenAI Tools Agent in LangChain is an intelligent agent that can autonomously use tools (functions, APIs, or external resources) to 
complete tasks. It works with OpenAI's function calling capability, allowing LLMs to decide when and how to use tools during execution.
'''

1) OPENAI TOOL AGENT (Able execute code)

In [ ]:
## Now convert the tools to Agents
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/openai_tools/
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [ ]:
'''
The AgentExecutor is a key component in LangChain that executes agents by managing their reasoning and tool usage. It takes an agent (decision-making system) and a set of tools (functions/APIs) and ensures that the agent properly calls tools, processes responses, and generates final outputs.

How AgentExecutor Works: 
- Receives a query from the user.
- The agent decides whether it needs to call a tool or generate a direct response.
- If a tool is needed, AgentExecutor calls the tool, gets the response, and feeds it back to the agent.
Repeats this process until a final answer is generated.
'''

In [16]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [ ]:
# We have used 2 tools (wiki, arxiv), now OpenAI_tools(which is an AI agent) desices which tool to use among these 2 bydefault.
# Here bydefault it uses wiki tool
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Get started by addi

{'input': 'Tell me about Langsmith',
 'output': 'Based on the output from the tool call, it appears that Langsmith is a platform that enables the development and deployment of AI applications, specifically those that utilize Large Language Models (LLMs). It seems to provide a range of tools and features to help developers and users build, test, and optimize their AI applications.\n\nSome of the key features mentioned include:\n\n1. Observability: Langsmith provides LLM-native observability to help developers understand and debug their AI applications.\n2. Evals: The platform allows users to build and run high-quality evaluations to test and optimize their AI applications.\n3. Prompt Engineering: Langsmith provides tools to enable and facilitate prompt engineering, which is the process of writing prompts to instruct LLMs on what to do.\n\nOverall, Langsmith appears to be a comprehensive platform that aims to make it easier for developers to build and deploy AI applications, particularly

In [ ]:
# # Here bydefault it uses wiki tool
agent_executor.invoke({"input":"What is machine learning"})

In [18]:
# # Here bydefault it uses arxiv tool
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" introduced the Transformer architecture, which has become a fundamental component of many state-of-the-art natural language processing models.

> Finished chain.


{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper "Attention Is All You Need" introduced the Transformer architecture, which has become a fundamental component of many state-of-the-art natural language processing models.'}

2) ReAct Agent

In [ ]:
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI

In [22]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

c:\Users\Lenovo\Documents\Data Science1\Gen AI\AgenticAI\3-Chatbots\myenv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [23]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [24]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
Thought: LangChain is a language chain model, which is a type of AI model that can integrate multiple language models to perform a variety of tasks. To answer this question, I think I should search for LangChain-related information.

Action: langsmith-search
Action Input: "LangChain"Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you t

{'input': 'what is LangChain?',
 'output': 'LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications.'}